In [36]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [37]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [55]:
hanam_book_loan = pd.read_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/경기도_하남시_공공도서관 대출 현황_20180808.csv', encoding='CP949')
icheon_book_loan = pd.read_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/경기도_이천시_공공도서관 도서 대출 현황_20200813.csv', encoding='CP949')
guro_book_loan = pd.read_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/서울특별시교육청 구로도서관 대출도서현황(반기).csv', encoding='CP949')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
hanam_book_loan.head()

,도서관명,도서명,국제표준도서번호(ISBN),분류기호(한국십진분류법),대출일시,데이터기준일자
0,신장도서관,최강의 식사 : 인생을 바꾸는 실리콘밸리식 완전무결 2주 다이어트,9.79E+12,594.14,2018-04-01 9:10,2018-08-08
1,신장도서관,보노보노처럼 살다니 다행이야,9.79E+12,818.00,2018-04-01 9:10,2018-08-08
2,신장도서관,모두 모두 정말 좋아,9.79E+12,375.10,2018-04-01 9:39,2018-08-08
3,신장도서관,매일매일 잘해요:바른 습관,9.79E+12,375.10,2018-04-01 9:54,2018-08-08
4,신장도서관,공룡,9.79E+12,375.10,2018-04-01 9:54,2018-08-08


In [57]:
icheon_book_loan.head()

,소장처,서명,등록번호,분류기호,대출일,대출시간
0,이천시립도서관,너무 재밌어서 잠 못 드는 세계사,EM0224341,909,2019-03-30,19:51:28
1,이천시립도서관,환경정책의 이해 =Environment·policy,EM0096170,539.91,2019-03-30,17:56:11
2,이천시립도서관,파브르 곤충기 .2,EM0169800,495.2,2019-03-30,17:56:11
3,이천시립도서관,한국 개미 (The)ants of Korea,EM0222961,495.8,2019-03-30,17:56:10
4,이천시립도서관,(새벽에 혼자 읽는) 주역인문학,EM0207838,141.2,2019-03-30,17:54:03


In [58]:
guro_book_loan.head()

,순번,등록번호,청구기호,서명(부록명),대출일,반납일,반납예정일
0,1,EMG000327806,600.04-ㅌ94ㄹ,리딩 아트 : 293개의 작품으로 만나는 책에 관한 예술,2018-07-01,2018-07-22,2018-07-22
1,2,EMG000328104,833.6-ㄱ216며,며느리를 그만두는 날 :가키야 미우 장편소설,2018-07-01,2018-07-22,2018-07-22
2,3,EMG000251796,031-ㅇ313ㄱ,(에듀윌) 공사·공단 일반상식 : 상식을 뛰어넘는 상식!,2018-07-01,2018-07-20,2018-07-22
3,4,EMG000298664,920-ㅋ592ㅅ-1,"새로운 서양 문명의 역사. 상, 문명의 기원에서 종교개혁까지",2018-07-01,2018-07-09,2018-07-15
4,5,EMG000282518,813.7-ㅇ646ㅇ-2,우여흔 :윤홍아 장편소설,2018-07-01,2018-07-30,2018-07-22


In [59]:
print("하남 결측치\n{}\n\n이천 결측치\n{}\n\n구로 결측치\n{}".format(hanam_book_loan.isnull().sum(), icheon_book_loan.isnull().sum(), guro_book_loan.isnull().sum()))

하남 결측치
도서관명                 0
도서명                  0
국제표준도서번호(ISBN)    1097
분류기호(한국십진분류법)        0
대출일시                 0
데이터기준일자              0
dtype: int64

이천 결측치
소장처     0
서명      1
등록번호    1
분류기호    0
대출일     1
대출시간    1
dtype: int64

구로 결측치
순번           0
등록번호         0
청구기호       479
서명(부록명)      1
대출일          0
반납일         86
반납예정일        0
dtype: int64


In [60]:
hanam_book_loan.columns, icheon_book_loan.columns, guro_book_loan.columns

(Index(['도서관명', '도서명', '국제표준도서번호(ISBN)', '분류기호(한국십진분류법)', '대출일시', '데이터기준일자'], dtype='object'),
 Index(['소장처', '서명', '등록번호', '분류기호', '대출일', '대출시간'], dtype='object'),
 Index(['순번', '등록번호', '청구기호', '서명(부록명)', '대출일', '반납일', '반납예정일'], dtype='object'))

In [61]:
preprocessed_hanam = hanam_book_loan.rename(columns={"대출일시":"대출일"})
preprocessed_icheon = icheon_book_loan.rename(columns={"서명":"도서명"})
preprocessed_guro = guro_book_loan.rename(columns={"서명(부록명)":"도서명"})

In [62]:
preprocessed_hanam.drop(['데이터기준일자', '도서관명'], axis=1, inplace=True)
preprocessed_icheon.drop(['대출시간', '소장처'], axis=1, inplace=True)
preprocessed_guro.drop(['순번', '반납예정일', '반납일'], axis=1, inplace=True)

In [63]:
preprocessed_hanam.head(1)

,도서명,국제표준도서번호(ISBN),분류기호(한국십진분류법),대출일
0,최강의 식사 : 인생을 바꾸는 실리콘밸리식 완전무결 2주 다이어트,9.79E+12,594.14,2018-04-01 9:10


In [64]:
preprocessed_icheon.head(1)

,도서명,등록번호,분류기호,대출일
0,너무 재밌어서 잠 못 드는 세계사,EM0224341,909,2019-03-30


In [66]:
preprocessed_guro.head(1)

,등록번호,청구기호,도서명,대출일
0,EMG000327806,600.04-ㅌ94ㄹ,리딩 아트 : 293개의 작품으로 만나는 책에 관한 예술,2018-07-01


In [68]:
preprocessed_hanam.to_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/preprocessed_hanam.csv')
preprocessed_icheon.to_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/preprocessed_icheon.csv')
preprocessed_guro.to_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/preprocessed_guro.csv')